In [7]:
# Manually implement CNN for CIFAR-10 dataset

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform to normalize the data and augment it
transform = transforms.Compose(
    [
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)


# Hyper-parameters
num_epochs = 20
num_classes = 10
batch_size = 32
learning_rate = 0.001

In [10]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root="./data/", train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data/", train=False, transform=transform
)

# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified


In [ ]:
# Convolutional neural network with following layers:
# CONV1: with Kernel size (3 ×3), In channels 3, Out channels 32.
# POOL1: Kernel size (2 ×2).
# CONV2: Kernel size (5 ×5), In channels 32, Out channels 64
# POOL2: Kernel size (2 ×2).
# CONV3: Kernel size (3 ×3), In channels 64, Out channels 64.
# FC1: Fully connected layer (also known as Linear layer) with 64 output neurons.
# FC2: Fully connected layer with 10 output neurons.
# Use ReLU as the activation function for all layers apart from the max-pooling layers, and the FC2layer. 
# You need to flatten the CONV3 output before passing as input to FC1. 
# Implement the forward and the backward passes. 
# Complete the code for the Conv2D, MaxPool2D, Linear, and ReLU layers and use these to implement the network.
# Complete the training code, 
# and train for 20 epochs using an Adam optimizer with learning rate 0.001 and batch size 32. 
# Use the categorical cross entropy loss as the loss function

class Conv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Conv2D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.randn(out_channels))
        
    def forward(self, x):
        # Add padding to the input tensor
        x = F.pad(x, (self.padding, self.padding, self.padding, self.padding))
        
        # Unfold the input tensor and reshape it
        x_unfolded = F.unfold(x, kernel_size=self.kernel_size, stride=self.stride)
        x_unfolded = x_unfolded.view(x.shape[0], self.in_channels, self.kernel_size, self.kernel_size, -1)
        
        # Perform the convolution using einsum
        x_conv = torch.einsum('nchwv, ochw -> nocv', x_unfolded, self.weight)
        
        # Add the bias and reshape the output tensor
        _, _, output_h, output_w = F.conv2d_input_size(x.shape, self.weight.shape, self.stride, self.padding)
        x_conv = x_conv.sum(dim=1).view(x.shape[0], self.out_channels, output_h, output_w)
        x_conv += self.bias.view(1, self.out_channels, 1, 1)
        
        return x_conv

class MaxPool2D(nn.Module):
    def __init__(self, kernel_size, stride=None, padding=0):
        super(MaxPool2D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride if stride is not None else kernel_size
        self.padding = padding

    def forward(self, x):
        # Implement the forward pass
        x = F.pad(x, (self.padding, self.padding, self.padding, self.padding))
        x_unfolded = F.unfold(x, kernel_size=self.kernel_size, stride=self.stride)
        x_unfolded = x_unfolded.view(x.shape[0], x.shape[1], self.kernel_size * self.kernel_size, -1)
        x = torch.max(x_unfolded, dim=2)[0]
        output_h, output_w = F.conv2d_output_size(x.shape[2:], (self.kernel_size, self.kernel_size), self.stride, self.padding)
        x = x.view(x.shape[0], x.shape[1], output_h, output_w)
        return x

class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))
        
    def forward(self, x):
        # Implement the forward pass
        x = x.view(x.shape[0], -1)
        x = torch.einsum('ni, io -> no', x, self.weight)
        x = x + self.bias.unsqueeze(0)
        return x
    
class ReLU(nn.Module):
    def __init__(self):
        super(ReLU, self).__init__()
        
    def forward(self, x):
        # Implement the forward pass
        x = torch.max(x, torch.zeros_like(x))
        return x

    
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = Conv2D(3, 32, 3, 1, 1)
        self.pool1 = MaxPool2D(2, 2)
        self.conv2 = Conv2D(32, 64, 5, 1, 2)
        self.pool2 = MaxPool2D(2, 2)
        self.conv3 = Conv2D(64, 64, 3, 1, 1)
        self.fc1 = Linear(64, 64)
        self.fc2 = Linear(64, num_classes)
        self.relu = ReLU()
        
    def forward(self, x):
        # Implement the forward pass
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
model = CNN(num_classes).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training and validation loss and accuracy tracking
epoch_train_loss = []
epoch_val_loss = []
epoch_val_acc = []

# Now, train the model for 20 epochs
total_step = len(train_loader)
for epoch in range(num_epochs):
    running_train_loss = 0
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        
    # Evaluate the model on the validation set
    running_val_loss = 0
    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    epoch_train_loss = running_train_loss / len(train_loader)
    epoch_val_loss = running_val_loss / len(test_loader)
    epoch_val_acc = val_correct / float(val_total)

    # Print the loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Train loss: {epoch_train_loss[-1]:.4f}, Validation loss: {epoch_val_loss[-1]:.4f}, Validation accuracy: {epoch_val_acc[-1]:.2f}%')

# Save the model checkpoint
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# Plot the loss curve
plt.figure()
plt.plot(range(1, num_epochs + 1), epoch_train_loss, label="Training Loss")
plt.plot(range(1, num_epochs + 1), epoch_val_loss, label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot the epoch vs. overall validation accuracy
plt.figure()
plt.plot(range(1, num_epochs + 1), epoch_val_acc)
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy (%)')
plt.title('Epoch vs. Overall Validation Accuracy')
plt.show()

In [ ]:
# Test the model for class-wise accuracy
model.eval()
with torch.no_grad():
    # Class-wise accuracy
    class_correct = [0] * num_classes
    class_total = [0] * num_classes
    for images, labels in test_loader:
        # Run the forward pass
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        # Collect the correct predictions for each class
        c = (predicted == labels)
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
    
    # Print the accuracy for each class
    for i in range(num_classes):
        print(f'Accuracy of {classes[i]}: {100 * class_correct[i] / class_total[i]}%')
